In [ ]:
!pip install scikit-learn opencv-python matplotlib seaborn pandas pyyaml
!pip install torch torchvision
!pip install -U ultralytics "ray[tune]"
!pip install roboflow
!pip install wandb

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="XXXXXXXXXXXX")
project = rf.workspace("capstonedesign-xkejj").project("banner_seg_detection")
version = project.version(3)
dataset = version.download("yolov11")
                

print("Dataset location:", dataset.location)

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to banner_seg_detection-3 in yolov11:: 100%|██████████| 7962/7962 [00:32<00:00, 244.04it/s]

Dataset location: /workspace/banner_seg_detection-3


In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
Device name: NVIDIA RTX A6000


In [4]:
from ultralytics import YOLO
from ray import tune
import wandb
import os

WARNING ⚠️ user config directory '/root/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD.Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
wandb.login(key="XXXXXXXXXXXXXXXX")

In [9]:
print("하위 폴더:", os.listdir("/workspace/banner_seg_detection-3"))

하위 폴더: ['valid', 'train', 'test', 'data.yaml', 'README.roboflow.txt', 'README.dataset.txt']


In [10]:
import os

os.environ["WANDB_PROJECT"] = "BANner_it_Tuning"
os.environ["WANDB_NAME"] = "yolo11s-seg-tuning"

In [11]:
model = YOLO("yolo11s-seg.pt") 

100%|██████████| 19.7M/19.7M [00:00<00:00, 106MB/s] 


In [14]:
!nvidia-smi      # GPU 수 확인
!lscpu           # CPU 코어 수 확인

Mon May 19 13:45:37 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               On  |   00000000:4F:00.0 Off |                    0 |
| 30%   29C    P8             25W /  300W |       4MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
result_grid = model.tune(
    data="workspace/banner_seg_detection-3/data.yaml",  
    space={
        "lr0": tune.loguniform(1e-4, 5e-3),
        "momentum": tune.uniform(0.75, 0.97),
        "weight_decay": tune.loguniform(1e-5, 1e-3),
        "batch": tune.choice([16, 32, 64]),
        "warmup_epochs": tune.choice([0, 2, 3, 5]),
    },
    epochs=30,
    iterations=20,   
    device=0,
    use_ray=True,
    gpu_per_trial=1
)

In [ ]:
experiment_path = "/kaggle/working/runs"
print(f"Loading results from {experiment_path}...")

restored_tuner = tune.Tuner.restore(experiment_path, trainable="/kaggle/working/banner_seg_detection-2")
result_grid = restored_tuner.get_results()

In [ ]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

In [ ]:
import pickle

with open(f"{experiment_path}/tuner.pkl", "rb") as f:
    data = pickle.load(f)

print(data["__trainable_name"])
